In [14]:
# This script is used to create Table 3 in the paper
# It requires running Table_3.sh to generate the results

import sys
sys.path.append('../../')
import pandas as pd
import matplotlib.pyplot as plt
from experiments.analysis.analysis_utils import find_results, load_result_from_timestamp
from experiments.benchmark import create_benchmark_datasets_if_not_exist
from tabulate import tabulate

# Change this to true only if you have access to the tacrolimus dataset
access_to_tacrolimus = False

create_benchmark_datasets_if_not_exist(dataset_description_path="../dataset_descriptions")

def create_df_row(dataset_name, model_name, n_basis):
    if n_basis is not None:
        results = find_results(dataset_name, model_name, filter_dict={"notes": f"n_basis={n_basis},rnn_type=lstm"})
    else:
        results = find_results(dataset_name, model_name)
    if len(results) == 0:
        print(f"No results found for {dataset_name} and {model_name} with n_basis={n_basis}")
        print("Make sure you run your experiments from ../run_scripts")
        # Return an empty row
        return pd.DataFrame(columns=["dataset", "method", "error_mean", "error_std"])
    elif len(results) > 1:
        print("Multiple results found for the given dataset and model")
        print("We take the last one but it may produce unexpected results")
    result = load_result_from_timestamp(results[-1])
    row = pd.DataFrame({"dataset": dataset_name,
                        "method": model_name,
                        "error_mean": result["results"][model_name]["mean"], 
                        "error_std": result["results"][model_name]["std"]}, index=[0])
    return row

# Get all timestamps

datasets =  [
            'synthetic_tumor_wilkerson_1',
            'airfoil_log',
            'flchain_1000',
            'stress-strain-lot-max-0.2',
            'sine_trans_200_20',
            'beta_900_20']

if access_to_tacrolimus:
    datasets.append("tacrolimus_visit_12")

baselines = [
    'TTS',
    'XGB',
    'RNN',
    'GAM',
    'SINDy',
    'DeltaTRNN',
    'CatBoost',
    'LGBM',
    'DecisionTree',
    'Linear',
    'PySR'
]

df = pd.DataFrame(columns=["dataset", "method", "error_mean", "error_std"])

for dataset in datasets:
    for baseline in baselines:
        if baseline == 'RNN' and dataset in ['airfoil_log','stress-strain-lot-max-0.2','tacrolimus_visit_12']:
           continue # RNN is not run on these datasets as it requires regular time intervals
        if dataset in ['sine_trans_200_20', 'beta_900_20']:
            n_basis = 5
        else:
            n_basis = 9
        if baseline == 'PySR':
            n_basis = None
        df = pd.concat([df, create_df_row(dataset, baseline, n_basis)], ignore_index=True)

# Reorder the columns
df = df[['method','dataset','error_mean','error_std']]

# Change dataset names
new_dataset_names = {
    "airfoil_log":"Airfoil",
    "celgene":"Celgene",
    "flchain_1000":"flchain",
    "stress-strain-lot-max-0.2":"Stress-Strain",
    "synthetic_tumor_wilkerson_1":"Tumor",
    "tacrolimus_visit_12":"Tacrolimus",
    "sine_trans_200_20": "Sine",
    "beta_900_20": "Beta",
}

df['dataset'] = df['dataset'].map(new_dataset_names)

# Change method names
new_method_names = {
    "TTS":"TIMEVIEW",
    "XGB":"XGB-T",
    "RNN":"RNN",
    "GAM":"GAM-T",
    "SINDy":"SINDy",
    "DeltaTRNN":"Dt-RNN",
    "CatBoost":"CatBoost-T",
    "LGBM":"LGBM-T",
    "DecisionTree":"DecisionTree-T",
    "Linear":"Linear-T",
    "PySR":"PySR"
}

df['method'] = df['method'].map(new_method_names)

# Pivot the dataframe to have datasets as columns and methods as rows
pivot_df = df.pivot_table(index='method', columns='dataset', values=['error_mean', 'error_std'])

# Switch the column levels and sort them
pivot_df = pivot_df.swaplevel(axis=1).sort_index(axis=1)

methods_in_order = ['RNN',"Dt-RNN","XGB-T",'LGBM-T','CatBoost-T','PySR','SINDy','Linear-T','DecisionTree-T','GAM-T','TIMEVIEW']

if access_to_tacrolimus:
    datasets_in_order = ["Airfoil", "flchain","Stress-Strain", "Tacrolimus", "Tumor", "Sine", "Beta"]
else:
    datasets_in_order = ["Airfoil", "flchain","Stress-Strain", "Tumor", "Sine", "Beta"]

pivot_df = pivot_df.loc[methods_in_order,:]

# Format the table cells to show mean and std as mean ± std
def format_mean_std(row):
    formatted_row = {}
    for dataset in datasets_in_order:
        mean, std = row[(dataset,'error_mean')], row[(dataset,'error_std')]
        formatted_row[dataset] = f"{mean:.2f} ± {std:.2f}"
    return pd.Series(formatted_row)

formatted_df = pivot_df.apply(format_mean_std, axis=1)

# Generate the LaTeX table
latex_table = tabulate(formatted_df, tablefmt='latex_booktabs', headers='keys', showindex=True, floatfmt=".2f")

# Print the LaTeX table
print(latex_table)

# Save the LaTeX table to a file
# with open("tables/Table_3.tex", "w") as f:
#     f.write(latex_table)


No results found for synthetic_tumor_wilkerson_1 and TTS with n_basis=9
Make sure you run your experiments from ../run_scripts
No results found for synthetic_tumor_wilkerson_1 and XGB with n_basis=9
Make sure you run your experiments from ../run_scripts
No results found for synthetic_tumor_wilkerson_1 and RNN with n_basis=9
Make sure you run your experiments from ../run_scripts
No results found for synthetic_tumor_wilkerson_1 and GAM with n_basis=9
Make sure you run your experiments from ../run_scripts
No results found for synthetic_tumor_wilkerson_1 and SINDy with n_basis=9
Make sure you run your experiments from ../run_scripts
Multiple results found for the given dataset and model
We take the last one but it may produce unexpected results
Multiple results found for the given dataset and model
We take the last one but it may produce unexpected results
Multiple results found for the given dataset and model
We take the last one but it may produce unexpected results
Multiple results foun

KeyError: "['RNN', 'XGB-T', 'SINDy', 'GAM-T'] not in index"